In [1]:
include("preload.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Normal/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["fsp_1e3"]
include("preload.jl")


2018-08-30 12:12:53.036345: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-08-30 12:12:53.036360: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-08-30 12:12:53.036364: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-08-30 12:12:53.036367: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


In [4]:
mutable struct Counter
    i::Int
    I::Int
    states::Array{Array{Float64}}
    daccs::Array{Float64}
    dsteers::Array{Float64}
    daccs_out::Array{Float64}
    dsteers_out::Array{Float64}
    done::Bool
end
function Counter(N::Int)
    return Counter(0,N,[],[],[],[],[],false)
end

Counter

In [5]:
function handle_action!(pc,scene,models,roadway,counter,ad_num)
    state = get_observation(scene,models,roadway)
    append!(counter.states,[state])
    
    action1_arr = pc[:getAction](state,1)
    action1_arr = tanh.(action1_arr)

    dacc = 0.0
    dsteer = 0.0
    if ad_num == 1
        action2_arr = pc[:getAction](state,2)
    elseif ad_num == 2
        action2_arr = pc[:getAction_regressor](state,2)
    end
    
    append!(counter.daccs_out,action2_arr[1])
    append!(counter.dsteers_out,action2_arr[2])
    
    action2_arr = tanh.(action2_arr)
    dacc = action2_arr[1]
    dsteer = action2_arr[2]
    
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    
    append!(counter.daccs,dacc)
    append!(counter.dsteers,dsteer)
    
    bump = simulate_action!(action,scene,models,roadway)
    reward = reward_fn(action,scene,models,roadway)
    
    if bump
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.i > counter.I
            counter.done = true
        end
    end
end

handle_action! (generic function with 1 method)

In [7]:
log_path = string(path,"Disturbance/")
scene, models, roadway = initialize_env()
for pname in pnames
    println(pname)
    policy_path = string(path,"Policy/",pname,".pkl")
    pc[:reset_policy](policy_path=policy_path)
    N = 9

    counter = Counter(N)
    rand_ego!(scene,models,roadway)
    pc[:resetPolicy](1)
    while !counter.done
        handle_action!(pc,scene,models,roadway,counter,1)
    end
    writedlm(string(log_path,pname,"_states",".txt"),counter.states)
    writedlm(string(log_path,pname,"_daccs",".txt"),counter.daccs)
    writedlm(string(log_path,pname,"_dsteers",".txt"),counter.dsteers)
    writedlm(string(log_path,pname,"_daccs_out",".txt"),counter.daccs_out)
    writedlm(string(log_path,pname,"_dsteers_out",".txt"),counter.dsteers_out)
end

fsp_1e3
